In [1]:
%sh
wget -P /tmp https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-12.csv
wget -P /tmp https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2018-12.csv

In [2]:
dbutils.fs.cp("file:///tmp/yellow_tripdata_2018-12.csv", "/mnt/datalake/")
dbutils.fs.cp("file:///tmp/green_tripdata_2018-12.csv", "/mnt/datalake/")

In [3]:
%fs ls /mnt/datalake/

path,name,size
dbfs:/mnt/datalake/GreenTaxiTripData_201812.csv,GreenTaxiTripData_201812.csv,62056490
dbfs:/mnt/datalake/PaymentTypes.json,PaymentTypes.json,277
dbfs:/mnt/datalake/RateCodes.csv,RateCodes.csv,139
dbfs:/mnt/datalake/RateCodes.json,RateCodes.json,252
dbfs:/mnt/datalake/TaxiZones.csv,TaxiZones.csv,12322
dbfs:/mnt/datalake/yellow_tripdata_2018-12.csv,yellow_tripdata_2018-12.csv,721522221


In [4]:
from pyspark.sql.functions import year, month, dayofmonth, unix_timestamp, round, when, col

In [5]:
yellow_taxi_trip_df = spark \
    .read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/mnt/datalake/yellow_tripdata_2018-12.csv")

In [6]:
display(yellow_taxi_trip_df.describe("passenger_count", "trip_distance"))

summary,passenger_count,trip_distance
count,8173231,8173231
mean,1.5964102813195908,2.8926264215460553
stddev,1.233920232393633,3.764338945224816
min,0,0.0
max,9,602.3


In [7]:
filtered_yellow_taxi_trips = yellow_taxi_trip_df.where((yellow_taxi_trip_df['trip_distance'] > 0) & (yellow_taxi_trip_df['passenger_count'] > 0))

In [8]:
display(filtered_yellow_taxi_trips.describe("passenger_count", "trip_distance"))

summary,passenger_count,trip_distance
count,7997713,7997713
mean,1.6208786186751138,2.9159603939275436
stddev,1.229214263122363,3.771235678655414
min,1,0.01
max,9,602.3


In [9]:
print('before = ', filtered_yellow_taxi_trips.count())
filtered_yellow_taxi_trips = filtered_yellow_taxi_trips.dropna(subset=("PULocationID", "DOLocationID"))
print('after', filtered_yellow_taxi_trips.count())

before = 7997713
after 7997713

In [10]:
default_values_dict = {
  "payment_type": 5,
  "RateCodeId": 1
}

filtered_yellow_taxi_trips = filtered_yellow_taxi_trips.na.fill(default_values_dict)
#filtered_yellow_taxi_trips = filtered_yellow_taxi_trips.na.fill(value=0, subset=("PULocationID"))

In [11]:
print('before = ', filtered_yellow_taxi_trips.count())
filtered_yellow_taxi_trips = filtered_yellow_taxi_trips.drop_duplicates()
print('after', filtered_yellow_taxi_trips.count())

before = 7997713
after 7997713

In [12]:
print('before = ', filtered_yellow_taxi_trips.count())
filtered_yellow_taxi_trips = filtered_yellow_taxi_trips \
                              .where("tpep_pickup_datetime >= '2018-12-01' AND tpep_dropoff_datetime < '2019-01-01'")

print('after', filtered_yellow_taxi_trips.count())

before = 7997713
after 7995244

In [13]:
green_taxi_trip_df = spark \
    .read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("delimiter", "\t") \
    .csv("/mnt/datalake/green_tripdata_2018-12.csv")


In [14]:
display(green_taxi_trip_df.describe())

summary,VendorID lpep_pickup_datetime lpep_dropoff_datetime store_and_fwd_flag RatecodeID PULocationID DOLocationID passenger_count trip_distance fare_amount extra mta_tax tip_amount tolls_amount ehail_fee improvement_surcharge total_amount payment_type trip_type
count,685373
mean,null
stddev,null
min,"1 2018-12-01 00:00:27 2018-12-01 00:18:15 N 1 106 4 1 6.00 19 0.5 0.5 3.05 0 """" 0.3 23.35 1 1"
max,"2 2019-03-11 22:23:32 2019-03-11 22:34:23 N 1 182 242 1 1.87 9 0 0.5 0 0 """" 0.3 9.8 1 1"


###All the above cleanup oprations in a single step (Yellow taxi)

In [16]:
from pyspark.sql.functions import year, month, dayofmonth, unix_timestamp, round, when, col

yellow_taxi_trip_df = spark \
    .read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/mnt/datalake/yellow_tripdata_2018-12.csv")

default_values_dict = {
  "payment_type": 5,
  "RateCodeId": 1
}

print('before = ', yellow_taxi_trip_df.count())

filtered_yellow_taxi_trips = yellow_taxi_trip_df.where((yellow_taxi_trip_df['trip_distance'] > 0) & (yellow_taxi_trip_df['passenger_count'] > 0)) \
                                                .dropna(subset=("PULocationID", "DOLocationID")) \
                                                .na.fill(default_values_dict) \
                                                .drop_duplicates() \
                                                .where("tpep_pickup_datetime >= '2018-12-01' AND tpep_dropoff_datetime < '2019-01-01'")

print('after', filtered_yellow_taxi_trips.count())

before = 8173231
after 7995244

###All transformation oprations in a single step (Yellow taxi)

In [19]:
filtered_yellow_taxi_trips = filtered_yellow_taxi_trips \
                             .select( \
                              col('VendorID').alias('vendor_id'), \
                              col('tpep_pickup_datetime').alias('pickup_time'), \
                              col('tpep_dropoff_datetime').alias('dropoff_time'), \
                              col('trip_distance'), \
                              col('PULocationID').alias('pickup_loc'), \
                              col('DOLocationID').alias('dropoff_loc'), \
                              col('RatecodeID').alias('rate_code_id'), \
                              col('total_amount'), \
                              col('payment_type') \
                              ) \
                              \
                             .withColumn("trip_year", year('pickup_time')) \
                             .withColumn("trip_month", month('pickup_time')) \
                             .withColumn("trip_day", dayofmonth('pickup_time')) \
                              \
                              .withColumn("trip_duration", \
                                          round((unix_timestamp('dropoff_time') - unix_timestamp('pickup_time')) / 60) \
                                         ) \
                              \
                              .withColumn( \
                                          'trip_type', \
                                                      when(
                                                      col('rate_code_id') == 6, 'shared_trip'
                                                      )\
                                                      .when(
                                                      col('rate_code_id') == 1000, 'shared_trip' # There is no rate_code_id value of 1000. Have included for "switch case" example
                                                      )
                                                      .otherwise('solo_trip')
                                         ) \
                              .drop('rate_code_id')

In [20]:
'''
filtered_yellow_taxi_trips = filtered_yellow_taxi_trips \
                .withColumn("trip_year", year('pickup_time')) \
                .withColumn("trip_month", month('pickup_time')) \
                .withColumn("trip_day", dayofmonth('pickup_time')) \
                \
                .withColumn("trip_duration", \
                            round(unix_timestamp('pickup_time') - unix_timestamp('dropoff_time')) / 60 \
                           )

filtered_yellow_taxi_trips = filtered_yellow_taxi_trips.withColumn( \
                                                                  'trip_type', \
                                                                  when(
                                                                  col('SR_Flag') == 1, 'shared_trip'
                                                                  )\
                                                                  .when(
                                                                  col('SR_Flag') == 2, 'shared_trip'
                                                                  )
                                                                  .otherwise('solo_trip')


'''

###All the above cleanup oprations in a single step (green taxi)

In [22]:
# Note the "delimiter" option. In green taxi csv, commas are not the seperators. <tab> has been used as a seperator
green_taxi_trip_df = spark \
    .read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("delimiter", "\t") \
    .csv("/mnt/datalake/green_tripdata_2018-12.csv")

default_values_dict = {
  "payment_type": 5,
  "RateCodeId": 1
}

#print('before = ', green_taxi_trip_df.count())

filtered_green_taxi_trips = green_taxi_trip_df.where((green_taxi_trip_df['trip_distance'] > 0) & (green_taxi_trip_df['passenger_count'] > 0)) \
                                                .dropna(subset=("PULocationID", "DOLocationID")) \
                                                .na.fill(default_values_dict) \
                                                .drop_duplicates() \
                                                .where("lpep_pickup_datetime >= '2018-12-01' AND lpep_dropoff_datetime < '2019-01-01'")

#print('after', filtered_green_taxi_trips.count())


###All transformation oprations in a single step (green taxi)

In [24]:
filtered_green_taxi_trips = filtered_green_taxi_trips \
                             .select( \
                              col('VendorID').alias('vendor_id'), \
                              col('lpep_pickup_datetime').alias('pickup_time'), \
                              col('lpep_dropoff_datetime').alias('dropoff_time'), \
                              col('trip_distance'), \
                              col('PULocationID').alias('pickup_loc'), \
                              col('DOLocationID').alias('dropoff_loc'), \
                              col('RatecodeID').alias('rate_code_id'), \
                              col('total_amount'), \
                              col('payment_type') \
                              ) \
                              \
                             .withColumn("trip_year", year('pickup_time')) \
                             .withColumn("trip_month", month('pickup_time')) \
                             .withColumn("trip_day", dayofmonth('pickup_time')) \
                              \
                              .withColumn("trip_duration", \
                                          round((unix_timestamp('dropoff_time') - unix_timestamp('pickup_time')) / 60) \
                                         ) \
                              \
                              .withColumn( \
                                          'trip_type', \
                                                      when(
                                                      col('rate_code_id') == 6, 'shared_trip'
                                                      )\
                                                      .when(
                                                      col('rate_code_id') == 1000, 'shared_trip' # There is no rate_code_id value of 1000. Have included for "switch case" example
                                                      )
                                                      .otherwise('solo_trip')
                                         ) \
                              .drop('rate_code_id')

In [25]:
filtered_yellow_taxi_trips.createOrReplaceGlobalTempView('fact_yellow_taxi_trip_data')
filtered_green_taxi_trips.createOrReplaceGlobalTempView('fact_green_taxi_trip_data')

In [26]:
display(filtered_yellow_taxi_trips)

In [27]:
display(filtered_green_taxi_trips)

In [28]:
spark.conf.get("spark.sql.shuffle.partitions")

# If you partition the table based on a particular column, most likely all the rows that belong a particular "column value" will be stored as individual partitions in hdfs
# So, if there is a column that you will be querying on frequently, you should partition the table by that col

Out[31]: '200'

In [29]:
filtered_green_taxi_trips.write \
                              .option('header', 'true') \
                              .option('dateFormat', 'yyyy-MM-dd HH:mm:ss.S') \
                              .mode('overwrite') \
                              .csv('/mnt/datalake/dimensional_model/facts/green_taxi_fact.csv')
# other options for mode are 'append', 'overwrite', 'ignore' and 'error' https://kontext.tech/column/spark/357/save-dataframe-as-csv-file-in-spark

In [30]:
filtered_green_taxi_trips.write \
                          .option('header', 'true') \
                          .option('dateFormat', 'yyyy-MM-dd HH:mm:ss.S') \
                          .mode('ignore') \
                          .parquet('/mnt/datalake/dimensional_model/facts/green_taxi_fact.parquet')

In [31]:
green_taxi_trips_csv_df = spark.read \
                                .option('header', 'true') \
                                .csv('/mnt/datalake/dimensional_model/facts/green_taxi_fact.csv')

print(green_taxi_trips_csv_df.select('pickup_loc', 'dropoff_loc').distinct().count())

# you cannot use display() instead of print here because display can only be used to print spark dataframe objects
# since the .count() outputs a number, we used print 

25052

In [32]:
green_taxi_trips_parquet_df = spark.read \
                                .parquet('/mnt/datalake/dimensional_model/facts/green_taxi_fact.parquet')

print(green_taxi_trips_parquet_df.select('pickup_loc', 'dropoff_loc').distinct().count())

# reading data from parquet is much faster. Writing data to parquet is slower because the schema is stored in parquet file.
# If more read operations are anticipated on a file, parquet format is better and vice versa

25052

In [33]:
%sql

create database if not exists taxi_service_warehouse;

In [34]:
green_taxi_trip_df.write \
                  .mode('overwrite') \
                  .saveAsTable("taxi_service_warehouse.green_taxi_trips_managed")

In [35]:
%sql

select * from taxi_service_warehouse.green_taxi_trips_managed limit 10

VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type
2,2018-12-16 13:35:23,2018-12-16 13:54:54,N,5,77,91,1,4.02,15.0,0.0,0.5,0.0,0.0,null,0.0,15.5,1,2
2,2018-12-16 13:42:05,2018-12-16 13:45:16,N,1,74,42,1,0.79,4.5,0.0,0.5,0.0,0.0,null,0.3,5.3,2,1
2,2018-12-16 13:17:51,2018-12-16 13:30:00,N,1,82,95,2,1.64,9.5,0.0,0.5,2.06,0.0,null,0.3,12.36,1,1
2,2018-12-16 13:40:11,2018-12-16 13:51:36,N,1,82,83,2,3.35,13.0,0.0,0.5,0.0,0.0,null,0.3,13.8,2,1
2,2018-12-16 13:55:52,2018-12-16 14:20:24,N,1,260,170,2,5.23,19.5,0.0,0.5,0.0,0.0,null,0.3,20.3,2,1
2,2018-12-16 13:06:06,2018-12-16 13:14:52,N,1,74,236,3,1.92,8.5,0.0,0.5,2.5,0.0,null,0.3,11.8,1,1
2,2018-12-16 13:25:53,2018-12-16 13:31:16,N,1,41,42,3,0.95,6.0,0.0,0.5,0.0,0.0,null,0.3,6.8,2,1
2,2018-12-16 13:35:59,2018-12-16 13:40:12,N,1,42,74,3,0.78,5.0,0.0,0.5,0.0,0.0,null,0.3,5.8,1,1
2,2018-12-16 13:02:59,2018-12-16 13:26:31,N,1,226,196,1,3.9,17.5,0.0,0.5,0.0,0.0,null,0.3,18.3,1,1
2,2018-12-16 13:50:57,2018-12-16 15:03:04,N,5,95,186,1,10.13,30.82,0.0,0.5,0.0,5.76,null,0.0,37.08,1,2


In [36]:
%sql

describe taxi_service_warehouse.green_taxi_trips_managed;

col_name,data_type,comment
VendorID,int,null
lpep_pickup_datetime,string,null
lpep_dropoff_datetime,string,null
store_and_fwd_flag,string,null
RatecodeID,int,null
PULocationID,int,null
DOLocationID,int,null
passenger_count,int,null
trip_distance,double,null
fare_amount,double,null


In [37]:
%sql

describe extended taxi_service_warehouse.green_taxi_trips_managed;

col_name,data_type,comment
VendorID,int,null
lpep_pickup_datetime,string,null
lpep_dropoff_datetime,string,null
store_and_fwd_flag,string,null
RatecodeID,int,null
PULocationID,int,null
DOLocationID,int,null
passenger_count,int,null
trip_distance,double,null
fare_amount,double,null


In [38]:
%sql

drop table taxi_service_warehouse.green_taxi_trips_managed;

In [39]:
green_taxi_trip_df.write \
                  .mode('overwrite') \
                  .option('path', '/mnt/datalake/dimensional_model/facts/green_taxi_fact.parquet') \
                  .saveAsTable('taxi_service_warehouse.fact_green_taxi_trips_data')

In [40]:
%sql

describe extended taxi_service_warehouse.fact_green_taxi_trips_data

col_name,data_type,comment
VendorID,int,null
lpep_pickup_datetime,string,null
lpep_dropoff_datetime,string,null
store_and_fwd_flag,string,null
RatecodeID,int,null
PULocationID,int,null
DOLocationID,int,null
passenger_count,int,null
trip_distance,double,null
fare_amount,double,null


In [41]:
%sql

drop table taxi_service_warehouse.fact_green_taxi_trips_data

In [42]:
%sql
/* DO NOT HAVE <SPACE> BETWEEN % and SQL. Magic commands (commands that start with %) should not have space between characters */
CREATE TABLE IF NOT EXISTS taxi_service_warehouse.fact_green_taxi_trips_data
    USING parquet
    OPTIONS 
    (
        path "/mnt/datalake/dimensional_model/facts/green_taxi_fact.parquet"
    )
    
/* Another way of creating a table
CREATE TABLE IF NOT EXISTS taxi_service_warehouse.fact_green_taxi_trips_data
    USING parquet
    location "/mnt/datalake/dimensional_model/facts/green_taxi_fact.parquet"
*/